In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from googletrans import Translator
from pathlib import Path
import csv
import os

In [3]:
df01 = pd.read_csv('D:\emissionsmap\documentation\data\huella_biocapacidad.csv')

In [40]:
df02 = pd.read_csv('D:\emissionsmap\documentation\data\Temperatura_anomalia.csv', sep=';', low_memory=False, encoding='utf-8')

In [114]:
df03 = pd.read_csv(r'D:\emissionsmap\SubirData\Normalizados\Normal_primary_sub_energy_source.csv')

In [116]:
df04 = pd.read_csv('D:\emissionsmap\documentation\data\gases.csv')

In [29]:
df01.head(2)

,Año,Poblacion,Biocapacidad PerCap,Huella Ecologica PerCap,Tierra,IdAño
0,1961,3091840000,3.15,2.28,0.72381,f106b7f99d2cb30c3db1c3cc0fde9ccb
1,1962,3150419968,3.11,2.30,0.73955,95f6870ff3dcd442254e334a9033d349


In [41]:
df02.head(2)

,Entidad,Año,Anomalia de la temperatura media del promedio de 1961 a 1990,Limite superior (95% CI),Limite inferior (95% CI)
0,Global,1850,-0.373,-0.339,-0.425
1,Global,1851,-0.218,-0.184,-0.274


In [88]:
df04.head(5)

,year,Dif_co2
24,1959,NaN
25,1960,0.93
26,1961,0.73
27,1962,0.81
28,1963,0.54


In [63]:
df04 = df04.sort_values('year')

In [64]:
df04['Dif_co2'] = df04['Media co2'].diff()

In [65]:
df04 = df04[['year', 'Dif_co2']]

In [67]:
df02 = df02[['Año', 'Anomalia de la temperatura media del promedio de 1961 a 1990']]

In [69]:
df02.rename(columns={'Año':'year'}, inplace=True)

In [75]:
df_n = pd.merge(df02, df04[['year' ,'Dif_co2']], on=['year'], how='left')

In [78]:
df_n = df_n[df_n['year'] >= 1960]

In [80]:
df_n = pd.merge(df02, df01[[]])

Index(['year', 'Anomalia de la temperatura media del promedio de 1961 a 1990',
       'Dif_co2'],
      dtype='object')

In [101]:
df01.rename(columns={'Año':'year'}, inplace=True)

In [121]:
df01['dif_huel'], df01['dif_biocap'] = df01['Huella Ecologica PerCap'].diff(), df01['Biocapacidad PerCap'].diff()

In [122]:
df_pr = pd.merge(df_n, df01[['year','Huella Ecologica PerCap', 'Biocapacidad PerCap']], on=['year'], how='left')

In [123]:
df_pr.set_index('year', inplace=True)

In [111]:
df_pr.fillna(0, inplace=True)

In [119]:
df_pru = df_n.set_index('year')

In [86]:
import plotly.offline as py
col=['Anomalia de la temperatura media del promedio de 1961 a 1990', 'Dif_co2']
py.iplot([{
    'x':df_pru.index,
    'y':df_pru[col],
    'name': col
} for col in  df_pru.columns], filename='cufflinks/multiple-lines-on-same-chart')

In [129]:
df_pr2 = df_pr[['Huella Ecologica PerCap', 'Biocapacidad PerCap']]

In [130]:
import plotly.offline as py
col=['Huella Ecologica PerCap', 'Biocapacidad PerCap']
py.iplot([{
    'x':df_pr2.index,
    'y':df_pr2[col],
    'name': col
} for col in  df_pr2.columns], filename='cufflinks/multiple-lines-on-same-chart')

In [124]:
import plotly.offline as py
col=['Anomalia de la temperatura media del promedio de 1961 a 1990', 'Dif_co2', 'Huella Ecologica PerCap', 'Biocapacidad PerCap']
py.iplot([{
    'x':df_pr.index,
    'y':df_pr[col],
    'name': col
} for col in  df_pr.columns], filename='cufflinks/multiple-lines-on-same-chart')

In [126]:
df05 = pd.read_csv('D:\emissionsmap\documentation\data\huella_sostenible_por_pais.csv', sep=';')

In [2]:
df_06 = pd.read_csv('D:\emissionsmap\documentation\data\energy.csv')

In [3]:
df_06.drop(columns='Unnamed: 0', inplace=True)

In [303]:
df_top = df_06[(df_06['Country'] != 'World') & (df_06['Energy_type'] != 'all_energy_types')]

In [237]:
top_four = df_06[(df_06['Country'] != 'World') & (df_06['Energy_type'] == 'all_energy_types')]

In [201]:
top_four = top_four.groupby('Country')[['Energy_consumption' ,'Energy_production' ,'GDP' , 'Population', 'Energy_intensity_per_capita',
                                        'Energy_intensity_by_GDP', 'CO2_emission']].sum()
top_four.sort_values('CO2_emission', ascending=False, inplace=True)
top_four= top_four.reset_index()
df_top = df_top.groupby(['Country', 'Energy_type'])['CO2_emission'].mean()

In [209]:
types = ['all_energy_types', 'coal', 'natural_gas', 'petroleum_n_other_liquids', 'nuclear', 'renewables_n_other']
top_cinco = top_four['Country'].head(5).to_list()

In [118]:
df_ba = pd.read_csv('D:\emissionsmap\documentation\data\energy.csv')

In [129]:
df_ba = df_ba[(df_ba['Country']=='World') & (df_ba['Energy_type']!='all_energy_types')]
#df_ba.drop(columns='Unnamed: 0', inplace=True)

In [131]:
df_ba.head(2)

,Country,Energy_type,Year,Energy_consumption,Energy_production,GDP,Population,Energy_intensity_per_capita,Energy_intensity_by_GDP,CO2_emission,color
1,World,coal,1980,78.656134,80.114194,27770.910281,4.298127e+06,68.145921,10.547,1409.790188,"rgb(116, 255, 70)"
2,World,natural_gas,1980,53.865223,54.761046,27770.910281,4.298127e+06,68.145921,10.547,1081.593377,"rgb(250, 162, 57)"


In [113]:
from raceplotly.plots import barplot

In [146]:
my_raceplot = barplot(df_ba,  item_column='Energy_type', value_column='Energy_consumption', time_column='Year',top_entries=10)
fig=my_raceplot.plot(item_label = 'Tipo de Energías', value_label = 'Energy_consumption', frame_duration = 200, date_format='%Y',orientation='horizontal')

#Add chart title, format the chart, etc.
fig.update_layout(
      title='Production de energía',
      title_x=0.15,
      width=800,
      height=550,
      paper_bgcolor="whitesmoke",
      )

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\raceplotly\plots.py:295: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



sandybrown, seagreen, seashell, sienna, silver,
            skyblue, slateblue, slategray, slategrey, snow,
            springgreen, steelblue, tan, teal, thistle, tomato,
            turquoise, violet, wheat, white, whitesmoke,
            yellow, yellowgreen